<a href="https://colab.research.google.com/github/dgiri4132/adaptivegarchmodel/blob/main/adaptivegarchmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install arch


In [15]:
import yfinance as yf
import pandas as pd
tickers=[t.upper() for t in input('Enter Stocks tickers(separated by space like aapl msft tsla):').split()]
start_date=input('Enter the start date in yyyy-mm-dd format:')
end_date=input('Enter the end date in yyyy-mm-dd format:')
forecast_horizon=int(input('Enter number of days ahead that you want the projection: '))
price_data=yf.download(tickers,start_date,end_date,auto_adjust=False)['Adj Close']
return_data=price_data.pct_change().dropna()*100


Enter Stocks tickers(separated by space like aapl msft tsla):aapl
Enter the start date in yyyy-mm-dd format:2023-01-01
Enter the end date in yyyy-mm-dd format:2025-01-01
Enter number of days ahead that you want the projection: 10


[*********************100%***********************]  1 of 1 completed


In [26]:
#Analyzing residuals for best model to use
from arch import arch_model
from scipy.stats import skew, kurtosis
import numpy as np
import pandas as pd
def residual_analyze(return_series):
  model=arch_model(return_series,vol='GARCH',p=1,q=1,dist='normal')
  result=model.fit(disp='off')
  resid=result.resid
  cond_vol=result.conditional_volatility
  std_resid=resid/cond_vol
  s=skew(std_resid)
  k=kurtosis(std_resid,fisher=True)
  return s, k, result
model_results={}
vol_forecasts={}
for ticker in tickers:
  print(f'Analyzing {ticker}')
  s,k,_=residual_analyze(return_data[ticker])
  print(f'Skewness: {s:.3f}')
  print(f'Kurtosis: {k:.3f}')
  series=return_data[ticker]
  if abs(s)>0.5 and k>3:
    print('As both asymmetry and fat tails has been detected, we will be running EGARCH model with skewed t distribution')
    model=arch_model(series,vol='EGARCH',p=1,o=1,q=1,dist='skewt')
  elif abs(s)>0.5:
    print('only asymmetry has been detected, we will be running EGARCH with normal distribution')
    model=arch_model(series,vol='EGARCH',p=1,o=1,q=1,dist='normal')
  elif k>3:
    print('Only fat tails have been detected, we will be running GARCH with t-distribution.')
    model= arch_model(series,vol='GARCH',p=1,q=1,dist='t')
  else:
    print('Residuals look normal(no fat tails or skew), running GARCH with normal distribution.')
    model=arch_model(series,vol='GARCH',p=1,q=1,dist='normal')
  result=model.fit(disp='off')
  forecast=result.forecast(horizon=forecast_horizon)
  last_forecast=forecast.variance.iloc[-1]
  last_std=last_forecast.apply(np.sqrt)
  print(f'Forecasted standard deviation for next {forecast_horizon} day(s):')
  model_results[ticker]=result
  vol_forecasts[ticker]=last_std
vol_df=pd.DataFrame(vol_forecasts)
print(vol_df.round(4))

Analyzing AAPL
Skewness: 0.152
Kurtosis: 2.391
Residuals look normal(no fat tails or skew), running GARCH with normal distribution.
Forecasted standard deviation for next 10 day(s):
        AAPL
h.01  1.3009
h.02  1.3448
h.03  1.3501
h.04  1.3508
h.05  1.3508
h.06  1.3509
h.07  1.3509
h.08  1.3509
h.09  1.3509
h.10  1.3509
